In [1]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [2]:
##### EXTRACT THE DATA #####

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
users_path = 'eua-dataset\\users\\'
U = pd.read_csv(users_path + 'users-test.csv')

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
servers_path = 'eua-dataset\\edge-servers\\'
S = pd.read_csv(servers_path + 'serverstest.csv')


In [3]:
##### COVERAGE OF EACH SERVER #####

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

dij = []
radius = np.round(np.random.uniform(0.45,0.75,len(S)),3) #in Km
#radius = [0.03, 0.03, 0.03]

for i in range(len(S)):
  server_latitude = S.iloc[i]['LATITUDE']
  server_longitude = S.iloc[i]['LONGITUDE']
  temp = []
  dist_temp = []
  for j in range(len(U)):
    user_latitude = U.iloc[j]['Latitude']
    user_longitude = U.iloc[j]['Longitude']

    a = haversine(server_longitude, server_latitude, user_longitude, user_latitude) #distance between User and Server
    dist_temp.append(np.round(a,3))

  dij.append(dist_temp)


In [4]:
# Users
#U = list(range(0, 8))

# Servers
#S = list(range(0, 3))

# D = [CPU, disk I/O,.....]
D = np.zeros((4), dtype=int)

w1 = [1, 2, 1, 2]
w2 = [1, 2,	1, 2]
w3 = [5, 7,	6, 6]
w4 = [2, 3,	3, 4]
w5 = [5, 7,	6, 6]
w6 = [2, 3,	3, 4]
w7 = [1, 2, 1, 2]
w8 = [2, 3,	3, 4]
W = [w1, w2, w3, w4, w5, w6, w7, w8]


c1 = [4, 6, 6, 10]
c2 = [10, 17, 15, 16]
c3 = [10, 15, 11, 15]
C = [c1, c2, c3]


NORM = [LA.norm(w1), LA.norm(w2), LA.norm(w3), LA.norm(w4), LA.norm(w5), LA.norm(w6), LA.norm(w7), LA.norm(w8)]

index = sorted(range(len(NORM)), key=lambda a: NORM[a])


In [5]:
########## DECLARE THE MIP Solver ##########
solver = pywraplp.Solver.CreateSolver('SCIP')

In [6]:
########## DEFINE THE VARIABLES ##########
# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in range(len(U)):
    for i in range(len(S)):
        x[i, j] = solver.BoolVar(f'c[{i}][{j}]')

# y_i = True if edge server s_i is used to serve users
# y_i = False otherwise
y = {}
for i in range(len(S)):
    y[i] = solver.BoolVar(f'c[{i}]')

print('Number of variables =', solver.NumVariables())

Number of variables = 27


In [7]:
########## DEFINE THE CONSTRAINTS ##########

# For each server i
# the sum of memory requirements of the users allocated to server i
# does not exceed the memory capacity of i
for i in range(len(S)):
    for k in range(len(D)):
        solver.Add(
            solver.Sum([
                x[i, j] * W[j][k] for j in range(len(U))
            ]) <= C[i][k]*y[i]
        )


#Proximity constraint
for j in range(len(U)):
    for i in range(len(S)): 
        if dij[i][j]>= radius[i]:
            solver.Add(x[i,j]==0)
            
# Each user can be allocated just once
for j in range(len(U)):
    solver.Add(
        solver.Sum([x[i, j] for i in range(len(S))]) <= 1
    )

# Constraint: x, y [0,1] (Boolean Variables)
# has already been included in the definition of the variables

print('Number of constraints =', solver.NumConstraints())

1 0   dij:  0.034 radius:   0.03
2 1   dij:  0.057 radius:   0.03
0 3   dij:  0.041 radius:   0.03
2 3   dij:  0.064 radius:   0.03
0 4   dij:  0.043 radius:   0.03
2 4   dij:  0.044 radius:   0.03
0 5   dij:  0.035 radius:   0.03
0 6   dij:  0.057 radius:   0.03
1 6   dij:  0.049 radius:   0.03
0 7   dij:  0.039 radius:   0.03
1 7   dij:  0.044 radius:   0.03
Number of constraints = 31


In [8]:
########## DEFINE THE OBJECTIVE ##########

# Maximize the number of allocated users
objective = solver.Objective()
for j in range(len(U)):
    for i in range(len(S)):
        objective.SetCoefficient(x[i, j], 1)
objective.SetMaximization()

# Minimize the number of required edge servers
#objectiveMin = solver.Objective()

#for i in U:
 #   objectiveMin.SetCoefficient(y[i], 1)
#objectiveMin.SetMinimization()

In [9]:
########## CALL THE SOLVER ##########
status = solver.Solve()


########### DISPLAY THE SOLUTION ##########
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for j in range(len(U)):
        for i in range(len(S)):
            if int(x[i,j].solution_value()) == 1:
                print(f'User {j} has been allocated on node {i}')
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 8.0
User 0 has been allocated on node 0
User 1 has been allocated on node 0
User 2 has been allocated on node 2
User 3 has been allocated on node 1
User 4 has been allocated on node 1
User 5 has been allocated on node 1
User 6 has been allocated on node 2
User 7 has been allocated on node 2
